# DwC Occurrences. PSZMP dataset

**Puget Sound Zooplankton Monitoring Program dataset.** Alignment of dataset to Darwin Core (DwC) for NANOOS, https://www.nanoos.org. This data alignment work, including this Jupyter notebook, are described in the GitHub repository https://github.com/nanoos-pnw/obis-pszmp. See [README.md](https://github.com/nanoos-pnw/obis-pszmp/blob/main/README.md).   

Emilio Mayorga, https://github.com/emiliom

2/15,12,9, 2024

## Goals and scope of this notebook

Parse the source data and combine it with the `intermediate_DwC_taxonomy.csv` file created in the notebook `PSZMP-dwcTaxonomy.ipynb` to create the DwC "occurrence" file `DwC_occurrence.csv`. In this file, each record is an individual taxonomic occurrence observation from a sample, containing WoRM taxonomic information, sex and life stage. The original sex and life stage information in the source data (in the `Life History Stage` column, renamed to `life_history_stage`) was parsed, turned to all lower case,, manually corrected in some cases based on previous research (encoded in `data_preprocess.py` and `common_mappings.json`), and matched to standard vocabularies when applicable. This notebook also creates an intermediate file, `intermediate_DwC_occurrence_life_history_stage.csv`, containing a match up between the `OccurrenceID` created here and the `life_history_stage` entry, to be used later in the `PSZMP-dwceMoF.ipynb` notebook.

## Settings

In [1]:
from datetime import datetime
import json
from pathlib import Path
import uuid

import pandas as pd

from data_preprocess import read_and_parse_sourcedata

In [2]:
data_pth = Path(".")

Set to `True` when debugging. `csv` ﬁles will not be exported when `debug_no_csvexport = True`

In [3]:
debug_no_csvexport = False

## Process JSON file containing common mappings and strings

In [4]:
with open(data_pth / 'common_mappings.json') as f:
    common_mappings = json.load(f)

In [5]:
DatasetCode = common_mappings['datasetcode']
sex_dwciri_terms = common_mappings['sex_dwciri_terms']
life_stage_mappings = common_mappings['life_stage_mappings']

# TODO: Create two mappings from life_stage_mappings, for lifeStage and dwciri:lifeStage
lifeStage_mapping = {k:v[0] for k,v in life_stage_mappings.items()}
lifeStage_dwciri_terms = {k:v[1] for k,v in life_stage_mappings.items()}

## Pre-process data for Occurrence table

### Read and pre-processe the source data from Excel file

`usecols` defines the columns that will be kept and the order in which they'll be organized

In [6]:
# usecols = ['sample_code', 'species', 'life_history_stage', 'lhs_0', 'lhs_1']
usecols = ['Sample Code', 'Genus species_lc', 'Life History Stage_lc', 'lhs_0', 'lhs_1']

# occursource_df = read_and_parse_sourcedata(test_n_rows=1000)[usecols]
occursource_df = read_and_parse_sourcedata()[usecols]

# TODO: Rename more columns, if needed
occursource_df.rename(
    columns={
        'Sample Code':'sample_code',
        'Genus species_lc':'species',
        'Life History Stage_lc':'life_history_stage',
    },
    inplace=True
)

In [7]:
len(occursource_df)

153825

In [8]:
occursource_df.head()

,sample_code,species,life_history_stage,lhs_0,lhs_1
0,010218ELIV1151,acartia hudsonica,"male, adult",male,adult
1,010218ELIV1151,acartia longiremis,"female, adult",female,adult
2,010218ELIV1151,acartia longiremis,"male, adult",male,adult
3,010218ELIV1151,aetideus,"female, adult",female,adult
4,010218ELIV1151,aglantha digitale,medusa,medusa,None


## Merge resolved taxonomy from taxonomy csv

In [9]:
taxonomy_df = pd.read_csv(
    data_pth / "intermediate_DwC_taxonomy.csv"
)

In [10]:
occursource_df = occursource_df.merge(
    taxonomy_df, 
    how='inner', 
    left_on='species',
    right_on='verbatimIdentification'
)

In [11]:
len(occursource_df)

153825

In [12]:
occursource_df.head()

,sample_code,species,life_history_stage,lhs_0,lhs_1,scientificName,scientificNameID,taxonRank,kingdom,phylum,class,order,family,genus,scientificNameAuthorship,verbatimIdentification
0,010218ELIV1151,acartia hudsonica,"male, adult",male,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
1,010621SARAV1115,acartia hudsonica,copepodite,copepodite,None,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
2,010621SARAV1115,acartia hudsonica,"female, adult",female,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
3,010621SARAV1115,acartia hudsonica,"male, adult",male,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
4,010818SKETV1058,acartia hudsonica,"female, adult",female,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica


## Create and populate `occurrence_df` dataframe

### Add `occurrenceID`, `basisOfRecord`, `occurrenceStatus`

Assumes the `eventID` used for sample event types is the unmodified sample code.

In [13]:
occurrence_df = occursource_df.copy()

In [14]:
occurrence_df.insert(1, 'occurrenceID', [uuid.uuid4() for i in range(len(occurrence_df))])
occurrence_df.insert(2, 'basisOfRecord', 'MaterialSample')
occurrence_df.insert(3, 'occurrenceStatus', 'present')

occurrence_df.rename(columns={'sample_code':'eventID'}, inplace=True)

In [16]:
occurrence_df.head(10)

,eventID,occurrenceID,basisOfRecord,occurrenceStatus,species,life_history_stage,lhs_0,lhs_1,scientificName,scientificNameID,taxonRank,kingdom,phylum,class,order,family,genus,scientificNameAuthorship,verbatimIdentification
0,010218ELIV1151,32fcffc7-7f23-4144-8e18-293efaa61dde,MaterialSample,present,acartia hudsonica,"male, adult",male,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
1,010621SARAV1115,ebd8450c-17a4-49cd-a66a-d01430332161,MaterialSample,present,acartia hudsonica,copepodite,copepodite,None,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
2,010621SARAV1115,a4871f71-ba24-49d9-a2e8-6dd5797eed2e,MaterialSample,present,acartia hudsonica,"female, adult",female,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
3,010621SARAV1115,f6e9a172-6e5f-4c92-a13d-19d8d25eafc7,MaterialSample,present,acartia hudsonica,"male, adult",male,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
4,010818SKETV1058,929978d4-34bb-4320-8e6b-aeb328e74e4c,MaterialSample,present,acartia hudsonica,"female, adult",female,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
5,010821HCB004V1342,449d5680-ea3a-4fba-bbb6-a220396f5be5,MaterialSample,present,acartia hudsonica,copepodite,copepodite,None,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
6,010821HCB004V1342,7ef61c2b-7fad-41fe-bfb7-d4064ea7896f,MaterialSample,present,acartia hudsonica,"male, adult",male,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
7,010921Wat1V1237,cdc76e7e-9288-4d0b-912b-1af25ea95ca8,MaterialSample,present,acartia hudsonica,"female, adult",female,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
8,011315CAMV1330,940049c1-8148-408b-89af-dfa08a85af84,MaterialSample,present,acartia hudsonica,copepodite,copepodite,None,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
9,011315CAMV1330,4b265adf-ad7b-4cbb-b7d2-95d2ea6e46c4,MaterialSample,present,acartia hudsonica,"female, adult",female,adult,Acartia hudsonica,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica


### Map `life_history_stage` into `sex` and `lifeStage`

In [17]:
# dwc standard columns
occurrence_df.insert(
    4, 'sex', 
    occurrence_df['lhs_0'].apply(
        lambda s: s if s in ['female', 'male'] else 'indeterminate'
    )
)
occurrence_df.insert(
    5, 'lifeStage', 
    occurrence_df['lhs_0'].apply(lambda s: lifeStage_mapping[s])
)

# dwciri columns
nercvocabs_url = "http://vocab.nerc.ac.uk/collection/"
occurrence_df.insert(
    6, 'dwciri:sex', 
    occurrence_df['sex'].apply(lambda s: nercvocabs_url + sex_dwciri_terms[s])
)
occurrence_df.insert(
    7, 'dwciri:lifeStage', 
    occurrence_df['lhs_0'].apply(
        lambda s: None if lifeStage_dwciri_terms[s] is None 
        else nercvocabs_url + lifeStage_dwciri_terms[s]
    )
)

In [18]:
occurrence_df = (
    occurrence_df
    .sort_values(by=['eventID', 'scientificName', 'lifeStage', 'sex'])
    .reset_index(drop=True)
)

In [19]:
occurrence_df.head(5)

,eventID,occurrenceID,basisOfRecord,occurrenceStatus,sex,lifeStage,dwciri:sex,dwciri:lifeStage,species,life_history_stage,...,scientificNameID,taxonRank,kingdom,phylum,class,order,family,genus,scientificNameAuthorship,verbatimIdentification
0,010218ELIV1151,32fcffc7-7f23-4144-8e18-293efaa61dde,MaterialSample,present,male,adult,http://vocab.nerc.ac.uk/collection/S10/current...,http://vocab.nerc.ac.uk/collection/S11/current...,acartia hudsonica,"male, adult",...,urn:lsid:marinespecies.org:taxname:157664,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"Pinhey, 1926",acartia hudsonica
1,010218ELIV1151,c8152ecf-73c1-4440-9244-3d4b31c0aa0c,MaterialSample,present,female,adult,http://vocab.nerc.ac.uk/collection/S10/current...,http://vocab.nerc.ac.uk/collection/S11/current...,acartia longiremis,"female, adult",...,urn:lsid:marinespecies.org:taxname:104257,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"(Lilljeborg, 1853)",acartia longiremis
2,010218ELIV1151,796b6157-e575-428b-8629-d633372b1443,MaterialSample,present,male,adult,http://vocab.nerc.ac.uk/collection/S10/current...,http://vocab.nerc.ac.uk/collection/S11/current...,acartia longiremis,"male, adult",...,urn:lsid:marinespecies.org:taxname:104257,Species,Animalia,Arthropoda,Copepoda,Calanoida,Acartiidae,Acartia,"(Lilljeborg, 1853)",acartia longiremis
3,010218ELIV1151,978833ce-7232-48cb-a2bb-412674419543,MaterialSample,present,female,adult,http://vocab.nerc.ac.uk/collection/S10/current...,http://vocab.nerc.ac.uk/collection/S11/current...,aetideus,"female, adult",...,urn:lsid:marinespecies.org:taxname:104112,Genus,Animalia,Arthropoda,Copepoda,Calanoida,Aetideidae,Aetideus,"Brady, 1883",aetideus
4,010218ELIV1151,934387c5-cfbf-4f90-bab2-f90ae50dc32a,MaterialSample,present,indeterminate,medusae,http://vocab.nerc.ac.uk/collection/S10/current...,http://vocab.nerc.ac.uk/collection/S11/current...,aglantha digitale,medusa,...,urn:lsid:marinespecies.org:taxname:117849,Species,Animalia,Cnidaria,Hydrozoa,Trachymedusae,Rhopalonematidae,Aglantha,"(O. F. Müller, 1776)",aglantha digitale


## Export intermediate table for `life_history_stage` matching to csv

In [20]:
if not debug_no_csvexport:
    occurrence_df[['occurrenceID', 'life_history_stage']].to_csv(
        data_pth / 'intermediate_DwC_occurrence_life_history_stage.csv', index=False
    )

## Export `occurrence_df` to csv

In [21]:
len(occurrence_df)

153825

### Cleanup

In [22]:
occurrence_df.drop(columns=['species', 'life_history_stage', 'lhs_0', 'lhs_1'], inplace=True)

if not debug_no_csvexport:
    occurrence_df.to_csv(data_pth / 'aligned_csvs' / 'DwC_occurrence.csv', index=False)

## Package versions

In [23]:
print(
    f"{datetime.utcnow()} +00:00\n"
    f"pandas: {pd.__version__}"
)

2024-02-13 00:47:00.828160 +00:00
pandas: 1.5.3
